In [1]:
from happytransformer import HappyTextToText
happy_tt = HappyTextToText("T5", "prithivida/grammar_error_correcter_v1")
from happytransformer import TTSettings
top_k_sampling_settings = TTSettings(do_sample=True, top_k=50, temperature=0.7, min_length=1, max_length=10)

import pickle
import cv2
import mediapipe as mp
import numpy as np
import time  #Import the time module
import re
import gradio as gr
import base64
import tensorflow as tf
from tensorflow.keras.models import load_model

C:\Users\acer\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
05/20/2024 02:29:36 - INFO - happytransformer.happy_transformer -   Using device: cpu


In [2]:
# Set up MediaPipe models for holistic and face detection
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize the models
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.3)

# Labels for hand gesture recognition
labels_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'Iam', 4: 'del', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L',
                13: 'M', 14: 'N', 15: 'nothing', 16: 'O', 17: 'P', 18: 'Q', 19: 'R', 20: 'S', 21: ' ', 22: 'T', 23: 'U',
                24: 'V', 25: 'W', 26: 'X', 27: 'Y', 28: 'Z', 29: '0', 30: '1', 31: '2', 32: '3', 33: '4', 34: '5', 35: '6',
                36: '7', 37: '8', 38: '9', 39: 'Love' , 40: 'You' , 41: 'Ok' , 42: 'Thanks' , 43: 'How' , 44: 'are'}


# Load your Keras model
# Load the TFLite model
# Load your Keras model
model_dict = pickle.load(open('C:\\Users\\acer\\Downloads\\pj\\pj\\model.p', 'rb'))
model = model_dict['model']

In [3]:
def correct_sent(str):
     words=re.findall(r'\b\w+\b',str.lower())
     new =words[0];
     x=words[0]
     new+=" "
     for i in words:
          if x!= i:
             x=i
             new+=i+" "
     str=new+'.'
    #  return str
     result = happy_tt.generate_text(str, args=top_k_sampling_settings).text
     return result


In [4]:
#predict
def process_img(config):
 word = ""       
 data_aux = []
 x_ = []
 y_ = []
 # if it image path cv2.imread(config)
 frame = config

 H, W, _ = frame.shape
 frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

 # Perform face detection
 face_results = face_detection.process(frame_rgb)

 if face_results.detections:
      for detection in face_results.detections:
          bboxC = detection.location_data.relative_bounding_box
          ih, iw, _ = frame.shape
          bbox = (
              int(bboxC.xmin * iw),
              int(bboxC.ymin * ih),
              int(bboxC.width * iw),
              int(bboxC.height * ih),
          )

          cv2.rectangle(frame, bbox, (0, 255, 0), 2)

 # Perform hand and holistic detections
 results = holistic.process(frame_rgb)
 hand_results = hands.process(frame_rgb)
    
 # Process hand landmarks
 if hand_results.multi_hand_landmarks:
     for hand_landmarks in hand_results.multi_hand_landmarks:
         mp_drawing.draw_landmarks(
             frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())

         for i in range(len(hand_landmarks.landmark)):
             x = hand_landmarks.landmark[i].x
             y = hand_landmarks.landmark[i].y
             data_aux.append(x)
             data_aux.append(y)
             x_.append(x)
             y_.append(y)

     for i in range(len(hand_landmarks.landmark)):
         x = hand_landmarks.landmark[i].x
         y = hand_landmarks.landmark[i].y
         data_aux.append(x - min(x_))
         data_aux.append(y - min(y_))
          

     x1 = int(min(x_) * W) - 10
     y1 = int(min(y_) * H) - 10
     x2 = int(max(x_) * W) - 10
     y2 = int(max(y_) * H) - 10

     # Ensure that data_aux has the correct number of features
     if len(data_aux) != 84:
         print("Error: Incorrect number of features. Check your feature extraction process.")
     else:
     # Make predictions using the model
         prediction = model.predict([np.asarray(data_aux)])
         predicted_character = labels_dict[int(prediction[0])]
    
            
         if(len(predicted_character)==1):
          return predicted_character
                  
         else :   
          return (" " + predicted_character+" ")

        

 return ""

In [5]:
def predict_msg(video_path, interval=2):
    # Open the video file
    vidcap = cv2.VideoCapture(video_path)
    output_folder = 'output_frames'

    # Get the frame rate of the video
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    
    # Calculate the frame interval in terms of frames
    frame_interval = fps * interval
    
    # Initialize variables
    success, image = vidcap.read()
    count = 0
    frame_number = 0
    word=""
    # Loop through the video frames
    while success:
        # Only save frames at specified intervals
        if count % frame_interval == 0:
            # Save the frame
            cv2.imwrite(f"{output_folder}/frame_{frame_number}.jpg", image)
            word+=process_img(image)
            frame_number += 1
        
        # Read the next frame
        success, image = vidcap.read()
        count += 1
    
    # Release the video capture object
    vidcap.release()
    return correct_sent(word)
    #return word



In [6]:
# commented
# Example usage
video_path = "C:\\Users\\acer\\Desktop\\ok2.mp4"
output_folder = 'output_frames'
# # interval =  3 # seconds
print(predict_msg(video_path))

05/20/2024 02:30:12 - INFO - happytransformer.happy_transformer -   Moving model to cpu
05/20/2024 02:30:12 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


Iam del .


In [7]:
# commented
# img='C:\\Users\\lenovo\\Downloads\\test2.png'
# print(process_img(cv2.imread(img)))

In [8]:
# commented
# from flask import Flask, request, jsonify

# app = Flask(__name__)

# @app.route("/api", methods=["GET"])
# def predict():
#     d={}
#     inputchar=request.args['query']
#     answer=predict_msg(str(inputchar))
#     d['output']=answer
#     return d

# if __name__ == "__main__":
#     app.run()

In [9]:
# commented
# from flask import Flask, request, jsonify
# import requests
# app = Flask(__name__)
# @app.route("/api", methods=["POST"])
# def predict():
#     print("111111111111111111111111111111111")
#     inputchar=request.get_data()
#     vid=base64.b64decode(inputchar)
#     filename="C:\\Users\\acer\\Desktop\\salah.mp4"
#     with open(filename,'wb')as f:
#         f.write(vid)
#     print("2222222222222222222222222222222222")
#     response=predict_msg(filename)
#     print("333333333333333333333333333333333")
#     #return response
#     return "Prediction result for {}".format(filename)
 
# if __name__ == "__main__":
#     app.run()

In [10]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/api", methods=["GET"])
def predict():
    d={}
    inputchar=request.args['query']
    answer=predict_msg(inputchar)
    print(answer)
    d['output']=answer
    return d

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off
iam salah iam
How are you?
iam salah iam
iam salah . 
How are you?
iam ok .
I love you.


In [ ]:
# from flask import Flask, request, jsonify
# import cv2

# app = Flask(__name__)

# @app.route('/api', methods=['GET'])
# def predict():
#     video_path = request.args.get('query')
#     if not video_path:
#         return jsonify({'error': 'No video path provided'}), 400

#     try:
#         # فتح الفيديو باستخدام OpenCV
#         cap = cv2.VideoCapture(video_path)
        
#         # التحقق من أنه تم فتح الفيديو بنجاح
#         if not cap.isOpened():
#             return jsonify({'error': 'Could not open video file'}), 400

#         # الحصول على عدد الإطارات في الفيديو
#         frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#         # إغلاق الفيديو
#         cap.release()

#         # نتيجة المعالجة (مثال: عدد الإطارات في الفيديو)
#         result = f'The video has {frame_count} frames.'
        
#         return jsonify({'output': result})

#     except Exception as e:
#         return jsonify({'error': str(e)}), 500

# if __name__ == '__main__':
#     app.run(host='127.0.0.1', port=5000)
